<a href="https://colab.research.google.com/github/OziomaEunice/Sentiment_GPT/blob/develop3/LLaMA_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LLaMA For Sentiment Analysis**

In [1]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


## **Huggingface Login**

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-creden

## **Installing Bitsandbytes, Transformers, Accelerate, and Peft Libraries for LLaMA**

In [3]:
# install bitsandbytes library for its low usage of memory and prevent the system from crashing.
# install other libraries


#! pip install --upgrade bitsandbytes
#!pip install -q -U "transformers==4.36.2" "datasets==2.16.1" "accelerate==0.26.1" "bitsandbytes==0.42.0"
#! pip install -i https://test.pypi.org/simple/ bitsandbytes --upgrade
! pip uninstall bitsandbytes
! pip install bitsandbytes==0.39.0
#! pip install bitsandbytes==0.38.1
! pip install -q -U "torch==2.0.0" tensorboard
! pip install git+https://github.com/TimDettmers/bitsandbytes.git
! pip install --upgrade transformers accelerate
! pip install dataset trl
! pip install -q -U git+https://github.com/huggingface/peft.git

Found existing installation: bitsandbytes 0.39.0
Uninstalling bitsandbytes-0.39.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/bitsandbytes-0.39.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/bitsandbytes/*
Proceed (Y/n)? Y
  Successfully uninstalled bitsandbytes-0.39.0
  Using cached bitsandbytes-0.39.0-py3-none-any.whl (92.2 MB)
  Cloning https://github.com/TimDettmers/bitsandbytes.git to /tmp/pip-req-build-y51q89id
  Running command git clone --filter=blob:none --quiet https://github.com/TimDettmers/bitsandbytes.git /tmp/pip-req-build-y51q89id
  Resolved https://github.com/TimDettmers/bitsandbytes.git to commit f9eba9c8dd3ffc7d59036fbd16c2b0c498fd3041
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for bitsandbytes: filename=bitsandbytes-0.43.0.dev0-cp310-cp310-linux_x86_64.whl size=111472 sha256=64e18281cbaf74a0ec725987da6df8178b41c1b121ec3af9795385058ab

In [4]:
! pip show bitsandbytes

Name: bitsandbytes
Version: 0.43.0.dev0
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, torch
Required-by: 


In [5]:
! pip uninstall bitsandbytes
! pip install bitsandbytes==0.39.0
# ! pip install bitsandbytes==0.38.1

Found existing installation: bitsandbytes 0.43.0.dev0
Uninstalling bitsandbytes-0.43.0.dev0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/bitsandbytes-0.43.0.dev0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/bitsandbytes/*
    /usr/local/lib/python3.10/dist-packages/tests/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/tests/slow/__init__.py
    /usr/local/lib/python3.10/dist-packages/tests/slow/test_dpo_slow.py
    /usr/local/lib/python3.10/dist-packages/tests/slow/test_sft_slow.py
    /usr/local/lib/python3.10/dist-packages/tests/slow/testing_constants.py
    /usr/local/lib/python3.10/dist-packages/tests/test_best_of_n_sampler.py
    /usr/local/lib/python3.10/dist-packages/tests/test_core.py
    /usr/local/lib/python3.10/dist-packages/tests/test_data_collator_completion_only.py
    /usr/local/lib/python3.10/dist-packages/tests/test_dataset_formatting.py
    /usr/local/lib/python3.10/dist-packages/tests/test_ddpo_tr

## **Importing Libraries**

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
!apt-get update
!apt-get install cuda-toolkit-11-8
import os
os.environ["LD_LIBRARY_PATH"] += ":" + "/usr/local/cuda-11/lib64"
os.environ["LD_LIBRARY_PATH"] += ":" + "/usr/local/cuda-11.8/lib64"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cuda-toolkit-11-8 is already the newest version (11.8.0-1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [9]:
# import os
# os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.8/lib64:' + os.environ.get('LD_LIBRARY_PATH', '')

In [10]:
# !rm -r /usr/local/cuda
# !rm -r /usr/local/cuda-12.2
# ! rm -r /usr/local/cuda-12
# !rm -r /usr/local/cuda-11

In [11]:
# ! make CUDA_VERSION=DETECTED_CUDA_VERSION
# ! make CUDA_VERSION=118

In [12]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import torch
import torch.nn as nn
import transformers
import bitsandbytes as bnb
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer, setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


In [13]:
import nltk

In [14]:
# Download the stopwords dataset
nltk.download('stopwords')

# Download wordnet dataset
nltk.download('wordnet')

# Download punkt dataset
nltk.download('punkt')

# Get the set of English stop words
stop_words = set(stopwords.words('english'))

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
print(f"pytorch version {torch.__version__}")

pytorch version 2.0.0+cu117


In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")

working on cuda:0


## **Loading IMDb Dataset**

In [17]:
# since dataset is imported to my Google Colab (which will remain in here temporary)
# read excel file
df = pd.read_excel('/content/Tweets.xlsx') # for Twitter dataset
df2 = pd.read_excel('/content/IMDB_Dataset.xlsx') # for IMDb Movie Review dataset
# df = pd.read_excel('/content/Tweets.xlsx', names = ["text", "airline_sentiment"]) # for Twitter dataset
# df2 = pd.read_excel('/content/IMDB_Dataset.xlsx', names = ["review", "sentiment"]) # for IMDb Movie Review dataset

## **Preprocessing Dataset**

In [18]:
# drop columns that are not needed for processing data
# In this case, for the Twitter dataset
df = df.drop(columns=['tweet_id', 'airline_sentiment_gold', 'negativereason', 'negativereason_confidence', 'negativereason_gold', 'retweet_count', 'tweet_coord', 'tweet_created', 'tweet_location', 'user_timezone'])

### **Cleaning Dataset**

In [19]:
# clean (preprocess) the Twitter dataset
def cleanData1(text, min_word_length = 3):
  text = text.lower()
  text = re.sub(r'@[A-Za-z0-9]+', "", text) # this informs Python the the mentions in text must be substituted with an empty string
  text = re.sub(r'#', "", text) # removing #
  text = re.sub(r'RT[\s]+', "", text) # removing retweets
  text = re.sub(r'https?:\/\/\S+', "", text) # removing links
  text = ' '.join(word for word in text.split() if len(word) >= min_word_length and word not in stop_words) # Remove short words

  punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
  for p in punctuations:
      text = text.replace(p,'') #Removing punctuations

  # Lemmatize the words
  words = nltk.word_tokenize(text)
  lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
  text = ' '.join(lemmatized_words)

  return text

In [20]:
# clean (preprocess) the IMDb Movie Review dataset
def cleanData2(review, min_word_length = 3):
  review = review.lower()
  review = re.sub(r'@[A-Za-z0-9]+', "", review) # this informs Python the the mentions in text must be substituted with an empty string
  review = re.sub(r'#', "", review) # removing #
  review = re.sub(r'RT[\s]+', "", review) # removing retweets
  review = re.sub(r'https?:\/\/\S+', "", review) # removing links
  review = ' '.join(word for word in review.split() if len(word) >= min_word_length and word not in stop_words) # Remove short words

  punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
  for p in punctuations:
      review = review.replace(p,'') #Removing punctuations

  # Lemmatize the words
  words = nltk.word_tokenize(review)
  lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
  review= ' '.join(lemmatized_words)

  return review

In [21]:
# df['text'] = df['text'].apply(cleanData)

# Apply the cleanData function to all text columns in the dataframe
df = df.applymap(lambda x: cleanData1(x) if isinstance(x, str) else x)

In [22]:
df2 = df2.applymap(lambda x: cleanData2(x) if isinstance(x, str) else x)

## **Splitting into Training and Testing**

In [23]:
# split the dataset into training and testing sets,
# with 80% for training and 20% for testing

x_train = list()
x_test = list()

for sentiment in ["positive", "negative", "neutral"]:
    train, test = train_test_split(df[df.airline_sentiment==sentiment],
                                   train_size = 0.8,
                                   test_size = 0.2,
                                   random_state = 42)

    x_train.append(train)
    x_test.append(test)

In [24]:
# shuffle the training data in a replicable order => random_state=10
x_train = pd.concat(x_train).sample(frac=1, random_state=10)
x_test = pd.concat(x_test)

x_train = x_train.reset_index(drop=True)

In [25]:
# generate prompt for LLaMA
def generate_prompt(data_point, text, airline_sentiment):
    return f"""
            Analyze the sentiment of the reviews enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "positive" or "neutral" or "negative".

            [{data_point["text"]}] = {data_point["airline_sentiment"]}
            """.strip()

def generate_test_prompt(data_point, text):
    return f"""
            Analyze the sentiment of the news headline enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "positive" or "neutral" or "negative".

            [{data_point["text"]}] = """.strip()

In [26]:
# For Twitter
x_train = pd.DataFrame(x_train.apply(lambda x: generate_prompt(x, "text", "airline_sentiment"), axis=1),
                       columns=["text"])

# For testing Twitter & IMDb
y_true = x_test.airline_sentiment
x_test = pd.DataFrame(x_test.apply(lambda x: generate_test_prompt(x, "text" if "text" in x else "review"), axis=1),
                      columns=["text"])

In [27]:
# wrap the train data by the class from Hugging Face (https://huggingface.co/docs/datasets/index)
train_data = Dataset.from_pandas(x_train)

### **Creating a function to evaluate the results from the fine-tuned sentiment model**

In [28]:
def evaluate(y_true, y_pred):
    labels = ['positive', 'neutral', 'negative']
    mapping = {'positive': 1, 'neutral': 0, 'negative': -1}
    def map_func(x):
        return mapping.get(x, 2)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    accuracy_perc = accuracy * 100

    print(f"Accuracy: {accuracy_perc:.2f}%")
    print('---------------------------\n')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        accuracy_perc = accuracy * 100
        print(f'Accuracy for label {label}: {accuracy:.2f}%')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[-1, 0, 1])
    print('\nConfusion Matrix:')
    print(conf_matrix)

## **Downloading LLaMA model**

In [29]:
# use standard model and tokeniser from the huggingface transformer

model_name = "meta-llama/Llama-2-7b-hf" # llama model (using 7b parameter)

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model, tokenizer = setup_chat_format(model, tokenizer)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

### **Non Fine-tuning**

In [32]:
def predict(test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(x_test))):
        prompt = x_test.iloc[i]["text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 1,
                        do_sample=False,  # Use greedy decoding strategy
                        # temperature = 0.0,
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        else:
            y_pred.append("neutral")

    return y_pred

In [33]:
y_pred = predict(test, model, tokenizer)

100%|██████████| 2929/2929 [07:47<00:00,  6.26it/s]


In [34]:
evaluate(y_true, y_pred)

Accuracy: 21.17%
---------------------------

Accuracy for label 0: 0.89%
Accuracy for label 1: 0.14%
Accuracy for label -1: 0.00%

Classification Report:
              precision    recall  f1-score   support

          -1       1.00      0.00      0.00      1836
           0       0.20      0.89      0.33       620
           1       0.33      0.14      0.20       473

    accuracy                           0.21      2929
   macro avg       0.51      0.34      0.18      2929
weighted avg       0.72      0.21      0.10      2929


Confusion Matrix:
[[   2 1767   67]
 [   0  551   69]
 [   0  406   67]]
